<a href="https://colab.research.google.com/github/DexterWilliamson/school_work/blob/master/DexterWilliamson_Joining_and_Reshaping_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-19 10:57:59--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.138.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.138.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  16.7MB/s    in 13s     

2020-05-19 10:58:13 (14.8 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


In [0]:
#order_id, order_number, product_id, product_name, add_to_cart, Reorder
#order_products__prior, order_products__train, and products
import pandas as pd

products = pd.read_csv('products.csv')
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')


In [0]:
#First i'm adding the obvious 2 together
ordered = pd.concat([order_products__prior, order_products__train])

In [0]:
#Creating subsets to only have useful data
condition = ((products['product_name'] == 'Banana') | 
             (products['product_name'] == 'Bag of Organic Bananas') |
             (products['product_name'] == 'Organic Strawberries') |
             (products['product_name'] == 'Organic Baby Spinach') |
             (products['product_name'] == 'Organic Hass Avocado') |
             (products['product_name'] == 'Organic Avocado') |
             (products['product_name'] == 'Large Lemon') |
             (products['product_name'] == 'Strawberries') |
             (products['product_name'] == 'Limes') |
             (products['product_name'] == 'Organic Whole Milk') )

products_subset = products[condition]

In [0]:
#merging to get only the top 10
ordered = pd.merge(ordered, products_subset,
                   on='product_id')

In [0]:
#Checking
ordered['product_id'].unique()

array([21903, 13176, 47209, 24852, 47766, 21137, 27845, 47626, 16797,
       26209])

In [0]:
#Creating a list to find the total
ordered['Total?'] = ordered['add_to_cart_order'] * (ordered['reordered'] + 1)

In [0]:
#Grabbing the total
ordered['Total?'].groupby(ordered['product_name']).sum()

product_name
Bag of Organic Bananas    3610752
Banana                    4365487
Large Lemon               2135852
Limes                     2092541
Organic Avocado           2053022
Organic Baby Spinach      3286799
Organic Hass Avocado      2655980
Organic Strawberries      3539258
Organic Whole Milk        1393340
Strawberries              1769707
Name: Total?, dtype: int64

In [0]:
#I'm just getting rid of data I didn't use
#I could get rid of product_name since it's reredundant
#it just might be nice for different graphs and readablity
ordered = ordered.drop(['order_id', 'aisle_id', 'department_id'], axis=1)
ordered

,product_id,add_to_cart_order,reordered,product_name,Total?
0,21903,4,1,Organic Baby Spinach,8
1,21903,6,0,Organic Baby Spinach,6
2,21903,3,1,Organic Baby Spinach,6
3,21903,4,0,Organic Baby Spinach,4
4,21903,8,1,Organic Baby Spinach,16
...,...,...,...,...,...
2418309,26209,3,1,Limes,6
2418310,26209,2,1,Limes,4
2418311,26209,6,0,Limes,6
2418312,26209,22,1,Limes,44


This is a new table with different challenges

In [0]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy

#reset the index to not have treatment as the index

table2 = table2.reset_index()

#make the names become variables instead of column headers

tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])

#Rename for readability

tidy2 = tidy2.rename(columns={'index': 'trt',
                              'variable': 'name',
                              'value': 'result'})

#Get rid of treatment as the column is already named that
tidy2.trt = tidy2.trt.str.replace('treatment', '')
tidy2

#nevermind, a lot easier than I thought

,trt,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


In [0]:
# tidy -> table2

#put treatment back as the index, names are headers again, and result populates
#the dataframe

wide2 = tidy2.pivot_table(index='trt', columns='name', values='result')

wide2

#looks pretty bad

name,Jane Doe,John Smith,Mary Johnson
trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


In [0]:
import seaborn as sns
flights = sns.load_dataset('flights')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Flights Pivot Table
flights = flights.pivot_table(index='year', columns='month', values='passengers')
flights

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336
